In [8]:
import pandas as pd
import json 

df = pd.read_csv('../data/all-history-actions.csv')

groups = df.groupby(by="id")

bills = [i for i, g in groups]

f = open("../data/all-history-actions.json", "w")
f.write(df.T.to_json())
f.close()

In [ ]:
def define_bill_set(target_size):
  visited = set()
  graphable_bills = set()
  
  for b in bills: 
    if len(graphable_bills) > 1200:
      break 
    if b not in visited and b not in graphable_bills: 
      cloud = collectConnections(b) 
      if len(cloud) == target_size:
        graphable_bills.add(b)
      visited.update(cloud)
  
  return graphable_bills


graphable_bills = define_bill_set(6)


In [ ]:


def connectionGraph(bill):
  cloud = list(collectConnections(bill))
  cloud.append(bill)

  nodes = {}
  
  for bill in cloud:
    hist = getHistory(bill)
    connections = getReferencedBills(hist)
    connect_dates = []
    
    for h in hist: 
      pair = findBillNum.findall(h.action)
      for p in pair: 
        p_hist = getHistory(p)
        for ph in p_hist: 
          if ph.action.count(bill) > 0: 
             connect_dates.append({ 
                "x1": h.date, 
                "x2": ph.date or h.date, 
                "y1": h.id, 
                "y2": ph.id, 
                "action1": h.action, 
                "action2": ph.action
              })
        
    
      
      nodes[bill] = {'id': bill, 
              'connections': connections, 
              'connect_dates': connect_dates,
              'start': hist[0].date,
              'end': hist[-1].date,
              'history': hist
    }
    

  return nodes

# return: list of dicts that list id, all bills it's connected to, the array of actions
test_graph = connectionGraph('H3456')


In [ ]:


def search_graph(nodes): 
  nodes = copy.deepcopy(nodes)
  
  starting_node = max(nodes.values(), key=lambda x: x['end'])
  starting_node['depth'] = 0
  starting_node['parent'] = (starting_node['id'], starting_node['start'])
  
  queue = [starting_node] 
  visited = []
  result = []
  
  while len(queue) > 0: 
    n = queue.pop()
    
    sorted_nodes = sorted([nodes[bill_num] for bill_num in n['connections']], key=lambda x: x['id'])
    
    for node in sorted_nodes: 
      if node['id'] not in visited:
        node['depth'] = n['depth'] + 1
        node['parent'] = (n['id'], n['start'], n['end'])
        
        queue.append(node)
    visited.append(n['id'])
    result.append(n)
  
  
  
  return sorted(result, key=lambda x: x['id'])
  
  
result = search_graph(test_graph)

In [ ]:

def graph_cloud_by_dates(bill):
  cloud_nodes = connectionGraph(bill)
  graphable = search_graph(cloud_nodes)

  plt.figure(figsize=(20,5))
  
  
  for n in sorted(graphable, key=lambda x: x['id']):
    start = n['start']
    end = n['end']
    
    points = [n['id'], start, [n['id'], end]]    

    x = [parse(start), parse(end)]
    y = [n['id'], n['id']]
    
    plt.plot(x , y , '>-', linewidth=2, label=n["id"])
  
    
  plt.legend()
  plt.show()


# graph_cloud_by_dates("S3030")


In [ ]:

def d3_data(bill):
  cloud_nodes = connectionGraph(bill)
  graphable = search_graph(cloud_nodes)

  node_index = ['index', 'id', 'connections', 'connect_dates', 'start', 'end', 'history', 'depth', 'parent']
  action_index = ['index', 'over_index', 'id', 'action', 'branch', 'date']
  
  
  df_bills = pd.DataFrame(graphable)
  
  
  bill = []
  start_date = []
  
  data = []
  
  
  for g in df_bills.itertuples(): 
    g_series = pd.Series(g, index=node_index)
    
    connected_dates = g_series.connect_dates
    history_dates = pd.Series(g.history)
  
    d = {'id': g_series.id, 
        'start': g_series.start, 
        'end': g_series.end, 
        'connect_dates': connected_dates,
        'depth': g_series.depth, 
        'parent': g_series.parent,
        'history': [{"date": g.date, "action": g.action} for g in history_dates]
    }
    data.append(d)
  
  return json.dumps(data)

    
    
dump = json.dumps(json.loads(d3_data("H1")))


In [ ]:

print(dump)